In [19]:
from fastai import *
from fastai.vision import *
from fastai.metrics import *
from numpy.random import randint

In [2]:
labels_file = 'data/train.txt'

def getSpeedList(file):
    with open(file) as f:
        lines = [line.rstrip('\n') for line in f]

    return lines

labels = getSpeedList(labels_file)

In [61]:
path_img = 'vidCaps/'
fnames = get_image_files(path_img)

In [4]:
print(f'Labels: {len(labels)}\nFiles:  {len(fnames)}')

Labels: 20400
Files:  19623


In [5]:
df = pd.DataFrame(columns = ['name', 'label'])
df

,name,label


In [6]:
for i, file in enumerate(fnames):
    speed = labels[i]
    df.loc[i] = [str(file), float(speed)]
#     df[i] = [file, speed]

In [7]:
df.to_csv(r'vidCaps/labels.csv', index=False)

In [12]:
df = pd.read_csv('vidCaps/labels.csv')
df.head()

,name,label
0,vidCaps/28.105569_frame-0.jpg,28.105569
1,vidCaps/28.105569_frame-1.jpg,28.105569
2,vidCaps/28.106527_frame-2.jpg,28.106527
3,vidCaps/28.130404_frame-3.jpg,28.130404
4,vidCaps/28.109243_frame-4.jpg,28.109243


In [9]:
path = '/'

In [10]:
bs = 64
np.random.seed(42)

In [22]:
test = 'vidCaps/28.109243_frame-4.jpg'

In [45]:
def get_float_labels(fname):
    delimiters = ['/', '_']
    regex = '|' .join(map(re.escape, delimiters))
    fname = re.split(regex, fname)
    
    return fname[1]

In [89]:
delimiters = ['/', '_']
regex = '|' .join(map(re.escape, delimiters))
func = lambda o: (re.split(regex, str(o[1])))

In [90]:
a = ImageList.from_folder(path_img)
l1 = a.label_from_func(func)

TypeError: 'PosixPath' object is not subscriptable